# 人工神经网络

  本网络数据集采用MNIST手写数字识别数据集(0-9)，输入为Xtrain(55000,784),标签为ytrain(55000,10)，其中每一个样本的标签为一个行向量，采用one-hot编码，如[0,1,0,0,0,0,0,0,0,0]代表为数字1。网络结构总共为3层，第一层为输入层，采用784个结点，第二层为隐藏层，采用500个结点，第三层为输出层，采用10个结点。


因数据量太大，采用随机梯度下降法（SGD）来更新参数，每次所需的batch从next_batch()方法获得（mnist数据集自带next_batch()方法，但本模型并没有使用）


训练优化方法有四个：


1.GradientDescentOptimizer


2.MomentumOptimizer


3.RMSpropOptimizer


4.AdamOptimizer



训练一个模型的方法为初始化一个annModel(**args)，参数分别为网络层结构，训练集样本，训练集标签，及batch大小（选传，默认为100）。

然后选用一个优化方法即可开始训练。

GradientDescentOptimizer需要传入的参数为学习率α，训练次数（选传，默认为10000）

MomentumOptimizer需要传入的参数为学习率α，β1（选传，默认为0.9），训练次数（选传，默认为10000）

RMSpropOptimizer需要传入的参数为学习率α，β2（选传，默认为0.999），epsilon（选传，默认为1e-8），训练次数（选传，默认为10000）

AdamOptimizer需要传入的参数为学习率α，β1（选传，默认为0.9），β2（选传，默认为0.999），epsilon（选传，默认为1e-8），训练次数（选传，默认为10000）

In [3]:
import numpy as np
import random
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [7]:
mnist=input_data.read_data_sets("MNIST_data",one_hot=True)
layerDims=[784,500,10]  #网络层结构
Xtrain=mnist.train.images  #训练集数据
ytrain=mnist.train.labels  #训练集标签

Xvalidate=mnist.validation.images  #验证集数据
yvalidate=mnist.validation.labels  #验证集标签

Xtest=mnist.test.images  #测试集数据
ytest=mnist.test.labels  #测试集标签

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [62]:
class annModel():
    def __init__(self,layerDims,Xtrain,ytrain,batch_size=100):
        self.Xtrain=Xtrain
        self.ytrain=ytrain
        self.data_index = [i for i in range(self.Xtrain.shape[0])]
        self.layerDims=layerDims
        self.batch_index=0
        self.batch_size=batch_size
        self.weights=[np.random.randn(layerDims[dim+1],layerDims[dim])*np.sqrt(2 / layerDims[dim]) for dim in range(len(layerDims)-1)]
        self.biases=[np.zeros((layerDims[dim],1)) for dim in range(1,len(layerDims))]
        self.caches={}



    def forward(self,X,predict=False):
        L=len(self.layerDims)
        A=X
        if not predict:
            self.caches['A0']=A
        active_func=self.relu
        for dim in range(1,L):
            Z=self.weights[dim-1].dot(A)+self.biases[dim-1]
            if dim==(L-1):
                active_func=self.softmax
            A=active_func(Z)
            if not predict:
                self.caches['Z'+str(dim)]=Z
                self.caches['A'+str(dim)]=A
        return A



    def relu(self,Z):
        return np.maximum(0,Z)


    def relu_back(self,dA,Z):
        dZ=dA.copy()
        dZ[Z<=0]=0
        return dZ


    def softmax(self,Z):
        return np.divide(np.exp(Z),np.sum(np.exp(Z),axis=0).reshape(1,-1))


    def softmax_back(self,A,y):
        return A-y


    def backward(self,A,y,method='GradientDescent',beta1=0.9,beta2=0.999,epsilon=1e-8,t=0):
        m=A.shape[1]
        L=len(self.layerDims)
        for dim in reversed(range(L-1)):
            if dim == (L-2):
                active_func = self.softmax_back
                dZ=active_func(A,y)
            else:
                active_func=self.relu_back
                dZ = active_func(dA, self.caches['Z'+str(dim+1)])
            dW=(1./m)*dZ.dot(self.caches['A'+str(dim)].T)
            db=(1./m)*np.sum(dZ,axis=1,keepdims=True)
            dA=self.weights[dim].T.dot(dZ)
            self.updateParams(dW,db,dim,method,beta1,beta2,epsilon,t)



    def updateParams(self,dW,db,dim,method,beta1,beta2,epsilon,t):
        if method=='GradientDescent':
            self.weights[dim]=self.weights[dim]-self.learning_rate*dW
            self.biases[dim]=self.biases[dim]-self.learning_rate*db
            
        elif method=='Momentum':
            Vdw=beta1*self.caches['Vdw' + str(dim)]+(1-beta1)*dW
            Vdb=beta1*self.caches['Vdb' + str(dim)]+(1-beta1)*db
            self.weights[dim] = self.weights[dim] - self.learning_rate * Vdw
            self.biases[dim] = self.biases[dim] - self.learning_rate * Vdb
            self.caches['Vdw' + str(dim)] = Vdw
            self.caches['Vdb' + str(dim)] = Vdb
            
        elif method=='RMSprop':
            Sdw = beta2 * self.caches['Sdw' + str(dim)] + (1 - beta2) * np.square(dW)
            Sdb = beta2 * self.caches['Sdb' + str(dim)] + (1 - beta2) * np.square(db)
            self.weights[dim] = self.weights[dim] - self.learning_rate * (dW/(np.sqrt(Sdw+epsilon)))
            self.biases[dim] = self.biases[dim] - self.learning_rate * (db/(np.sqrt(Sdb+epsilon)))
            self.caches['Sdw' + str(dim)] = Sdw
            self.caches['Sdb' + str(dim)] = Sdb
            
        elif method=='Adam':
            Vdw = beta1 * self.caches['Vdw'+str(dim)] + (1 - beta1) * dW
            Vdb = beta1 * self.caches['Vdb'+str(dim)] + (1 - beta1) * db
            Sdw = beta2 * self.caches['Sdw'+str(dim)] + (1 - beta2) * np.square(dW)
            Sdb = beta2 * self.caches['Sdb'+str(dim)] + (1 - beta2) * np.square(db)
            Vdw_correctd=Vdw/(1-pow(beta1,t))
            Vdb_corrected=Vdb/(1-pow(beta1,t))
            Sdw_corrected=Sdw/(1-pow(beta2,t))
            Sdb_corrected=Sdb/(1-pow(beta2,t))
            self.weights[dim] = self.weights[dim] - self.learning_rate * (Vdw_correctd / (np.sqrt(Sdw_corrected)+epsilon))
            self.biases[dim] = self.biases[dim] - self.learning_rate * (Vdb_corrected / (np.sqrt(Sdb_corrected) + epsilon))
            self.caches['Vdw'+str(dim)] = Vdw
            self.caches['Vdb'+str(dim)] = Vdb
            self.caches['Sdw'+str(dim)] = Sdw
            self.caches['Sdb'+str(dim)] = Sdb



    def GradientDescentOptimizer(self,learning_rate,training_steps=10000):
        self.learning_rate=learning_rate
        for step in range(training_steps):
            xs, ys = self.next_batch()
            xs=xs.T
            ys=ys.T
            AL=self.forward(xs)
            self.backward(AL,ys)
            if step %1000==0:
                accuracy=self.calcAcc(Xvalidate,yvalidate)*100
                loss=self.calcLoss(Xvalidate,yvalidate)
                print("after %d training steps, training accuracy on validate dataset is %f%%,loss is %f" % (step, accuracy,loss))
        test_accuracy=self.calcAcc(Xtest,ytest)*100
        print("training complete!Test accuracy is: ",test_accuracy)


        
    def MomentumOptimizer(self,learning_rate,beta1=0.9,training_steps=10000):
        self.learning_rate = learning_rate
        for l in range(len(self.layerDims)-1):
            self.caches['Vdw'+str(l)]=np.zeros_like(self.weights[l])
            self.caches['Vdb'+str(l)]=np.zeros_like(self.biases[l])
        for step in range(training_steps):
            xs, ys = self.next_batch()
            xs=xs.T
            ys=ys.T
            AL = self.forward(xs)
            self.backward(AL, ys,'Momentum',beta1=beta1)
            if step % 1000 == 0:
                accuracy = self.calcAcc(Xvalidate, yvalidate)*100
                loss = self.calcLoss(Xvalidate, yvalidate)
                print("after %d training steps, training accuracy on validate dataset is %g%%,loss is %f" % (step, accuracy, loss))
        test_accuracy=self.calcAcc(Xtest,ytest)*100
        print("training complete!Test accuracy is: ",test_accuracy)


        
    def RMSpropOptimizer(self,learning_rate,beta2=0.999,epsilon=1e-8,training_steps=10000):
        self.learning_rate = learning_rate
        for l in range(len(self.layerDims)-1):
            self.caches['Sdw'+str(l)]=np.zeros_like(self.weights[l])
            self.caches['Sdb'+str(l)]=np.zeros_like(self.biases[l])
        for step in range(training_steps):
            xs, ys = self.next_batch()
            xs=xs.T
            ys=ys.T
            AL = self.forward(xs)
            self.backward(AL, ys,'RMSprop',beta2=beta2,epsilon=epsilon)
            if step % 1000 == 0:
                accuracy = self.calcAcc(Xvalidate, yvalidate)*100
                loss = self.calcLoss(Xvalidate, yvalidate)
                print("after %d training steps, training accuracy on validate dataset is %g%%,loss is %f" % (step, accuracy, loss))
        test_accuracy=self.calcAcc(Xtest,ytest)*100
        print("training complete!Test accuracy is: ",test_accuracy)

        
        
    def AdamOptimizer(self,learning_rate,beat1=0.9,beta2=0.999,epsilon=1e-8,training_steps=10000):
        self.learning_rate = learning_rate
        for l in range(len(self.layerDims)-1):
            self.caches['Vdw'+str(l)]=np.zeros_like(self.weights[l])
            self.caches['Vdb'+str(l)]=np.zeros_like(self.biases[l])
            self.caches['Sdw'+str(l)]=np.zeros_like(self.weights[l])
            self.caches['Sdb'+str(l)]=np.zeros_like(self.biases[l])
        t=0
        for step in range(training_steps):
            t=t+1
            xs, ys = self.next_batch()
            xs=xs.T
            ys=ys.T
            AL = self.forward(xs)
            self.backward(AL, ys,'Adam',beta1=beat1,beta2=beta2,epsilon=epsilon,t=t)
            if step % 1000 == 0:
                accuracy = self.calcAcc(Xvalidate, yvalidate)*100
                loss = self.calcLoss(Xvalidate, yvalidate)
                print("after %d training steps, training accuracy on validate dataset is %g%%,loss is %f" % (step, accuracy, loss))
        test_accuracy=self.calcAcc(Xtest,ytest)*100
        print("training complete!Test accuracy is: ",test_accuracy)


        
    def next_batch(self):
        m=self.Xtrain.shape[0]
        start = min(self.batch_index, m)
        if start==m:
            self.batch_index=0
            start=self.batch_index
        if self.batch_index==0:  #一轮数据遍历完了，重新打乱数据
            random.shuffle(self.data_index)
        end=min(start+self.batch_size,m)
        xs=self.Xtrain[start:end]
        ys=self.ytrain[start:end]
        self.batch_index=end
        return xs,ys

    

    def calcAcc(self,X,Y):
        Y_=np.argmax(self.forward(X.T,predict=True),axis=0).reshape(-1,1)
        Y=np.argmax(Y,axis=1).reshape(-1,1)
        accuracy=np.sum(Y_==Y)/len(Y_)
        return accuracy

    

    def calcLoss(self,X,Y):
        Y_=np.log(self.forward(X.T,predict=True))
        loss=np.sum(np.multiply(Y_.T,(-1)*Y))/Y.shape[0]
        return loss

# 梯度下降法

In [13]:
myANN=annModel(layerDims,Xtrain,ytrain)
myANN.GradientDescentOptimizer(0.8)

after 0 training steps, training accuracy on validate dataset is 15.740000%,loss is 2.471154
after 1000 training steps, training accuracy on validate dataset is 97.020000%,loss is 0.092826
after 2000 training steps, training accuracy on validate dataset is 97.960000%,loss is 0.071078
after 3000 training steps, training accuracy on validate dataset is 98.080000%,loss is 0.069773
after 4000 training steps, training accuracy on validate dataset is 97.980000%,loss is 0.068978
after 5000 training steps, training accuracy on validate dataset is 98.140000%,loss is 0.068056
after 6000 training steps, training accuracy on validate dataset is 98.220000%,loss is 0.070721
after 7000 training steps, training accuracy on validate dataset is 98.160000%,loss is 0.069036
after 8000 training steps, training accuracy on validate dataset is 98.260000%,loss is 0.068284
after 9000 training steps, training accuracy on validate dataset is 98.220000%,loss is 0.070371
training complete!Test accuracy is:  98.32


# 对比一下tensorflow实现的（本机环境为python3.6,tensorflow-gpu1.8）

In [38]:
INPUT_NODE=784
OUTPUT_NODE=10
LEARNING_RATE_BASE=0.8
TRAINING_STEPS=10000
BATCH_SIZE=100


def forward(X,layerDims):
    AL=X
    L=len(layerDims)
    for dim in range(1,L):
        with tf.variable_scope(('layer'+str(dim)),reuse=tf.AUTO_REUSE):
            weights=tf.get_variable("weights",[layerDims[dim-1],layerDims[dim]],initializer=tf.truncated_normal_initializer(stddev=0.1))
            biases=tf.get_variable("biases",[layerDims[dim]],initializer=tf.constant_initializer(0.0))
            AL=tf.matmul(AL,weights)+biases
            if dim!=(L-1):
                AL=tf.nn.relu(AL)
    return AL


def train():
    X = tf.placeholder(tf.float32, shape=[None,INPUT_NODE], name="X_input")
    y = tf.placeholder(tf.float32, shape=[None,OUTPUT_NODE], name="y_input")
    h = forward(X, layerDims)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=tf.argmax(y,1))
    loss = tf.reduce_mean(cross_entropy)
    train_step=tf.train.GradientDescentOptimizer(LEARNING_RATE_BASE).minimize(loss)
    correct_prediction=tf.equal(tf.argmax(h,1),tf.argmax(y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))*100
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        validate_feed = {X: mnist.validation.images, y: mnist.validation.labels}
        test_feed = {X: mnist.test.images,y: mnist.test.labels}

        for step in range(TRAINING_STEPS):
            if step%1000==0:
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("after %d training steps,training accuracy on validae dataset is %g%%" % (step, validate_acc))
            xs,ys=mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_step,feed_dict={X:xs,y:ys})
        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print("training complete!Test accuracy is: ",test_acc)

train()

after 0 training steps,training accuracy on validae dataset is 12.48%
after 1000 training steps,training accuracy on validae dataset is 97.32%
after 2000 training steps,training accuracy on validae dataset is 97.88%
after 3000 training steps,training accuracy on validae dataset is 97.98%
after 4000 training steps,training accuracy on validae dataset is 98.22%
after 5000 training steps,training accuracy on validae dataset is 98.3%
after 6000 training steps,training accuracy on validae dataset is 98.3%
after 7000 training steps,training accuracy on validae dataset is 98.38%
after 8000 training steps,training accuracy on validae dataset is 98.34%
after 9000 training steps,training accuracy on validae dataset is 98.4%
training complete!Test accuracy is:  98.08


# Adam

In [44]:
myANN=annModel(layerDims,Xtrain,ytrain)
myANN.AdamOptimizer(0.001)

after 0 training steps, training accuracy on validate dataset is 18.9%,loss is 2.263199
after 1000 training steps, training accuracy on validate dataset is 96.98%,loss is 0.096090
after 2000 training steps, training accuracy on validate dataset is 97.7%,loss is 0.076520
after 3000 training steps, training accuracy on validate dataset is 97.54%,loss is 0.075833
after 4000 training steps, training accuracy on validate dataset is 98.36%,loss is 0.062417
after 5000 training steps, training accuracy on validate dataset is 98%,loss is 0.073629
after 6000 training steps, training accuracy on validate dataset is 97.98%,loss is 0.081794
after 7000 training steps, training accuracy on validate dataset is 98.14%,loss is 0.076164
after 8000 training steps, training accuracy on validate dataset is 98.28%,loss is 0.074135
after 9000 training steps, training accuracy on validate dataset is 98.08%,loss is 0.085459
training complete!Test accuracy is:  98.28


# 对比一下tensorflow实现的（本机环境为python3.6,tensorflow-gpu1.8）

In [45]:
INPUT_NODE=784
OUTPUT_NODE=10
LEARNING_RATE_BASE=0.001
TRAINING_STEPS=10000
BATCH_SIZE=100


def forward(X,layerDims):
    AL=X
    L=len(layerDims)
    for dim in range(1,L):
        with tf.variable_scope(('layer'+str(dim)),reuse=tf.AUTO_REUSE):
            weights=tf.get_variable("weights",[layerDims[dim-1],layerDims[dim]],initializer=tf.truncated_normal_initializer(stddev=0.1))
            biases=tf.get_variable("biases",[layerDims[dim]],initializer=tf.constant_initializer(0.0))
            AL=tf.matmul(AL,weights)+biases
            if dim!=(L-1):
                AL=tf.nn.relu(AL)
    return AL


def train():
    X = tf.placeholder(tf.float32, shape=[None,INPUT_NODE], name="X_input")
    y = tf.placeholder(tf.float32, shape=[None,OUTPUT_NODE], name="y_input")
    h = forward(X, layerDims)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=tf.argmax(y,1))
    loss = tf.reduce_mean(cross_entropy)
    train_step=tf.train.AdamOptimizer(LEARNING_RATE_BASE).minimize(loss)
    correct_prediction=tf.equal(tf.argmax(h,1),tf.argmax(y,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))*100
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        validate_feed = {X: mnist.validation.images, y: mnist.validation.labels}
        test_feed = {X: mnist.test.images,y: mnist.test.labels}

        for step in range(TRAINING_STEPS):
            if step%1000==0:
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("after %d training steps,training accuracy on validae dataset is %g%%" % (step, validate_acc))
            xs,ys=mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_step,feed_dict={X:xs,y:ys})
        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print("training complete!Test accuracy is: ",test_acc)

train()

after 0 training steps,training accuracy on validae dataset is 10.92%
after 1000 training steps,training accuracy on validae dataset is 97.32%
after 2000 training steps,training accuracy on validae dataset is 97.74%
after 3000 training steps,training accuracy on validae dataset is 98.06%
after 4000 training steps,training accuracy on validae dataset is 98.36%
after 5000 training steps,training accuracy on validae dataset is 98.02%
after 6000 training steps,training accuracy on validae dataset is 98.16%
after 7000 training steps,training accuracy on validae dataset is 98.38%
after 8000 training steps,training accuracy on validae dataset is 98.3%
after 9000 training steps,training accuracy on validae dataset is 98.22%
training complete!Test accuracy is:  98.16


# 顺便跑下实现的Momentum和RMSprop，篇幅原因就不对比tensorflow了

# Momentum

In [53]:
myANN=annModel(layerDims,Xtrain,ytrain)
myANN.MomentumOptimizer(0.8)

after 0 training steps, training accuracy on validate dataset is 16.16%,loss is 2.266412
after 1000 training steps, training accuracy on validate dataset is 97.46%,loss is 0.090930
after 2000 training steps, training accuracy on validate dataset is 97.56%,loss is 0.085476
after 3000 training steps, training accuracy on validate dataset is 98.06%,loss is 0.067338
after 4000 training steps, training accuracy on validate dataset is 98.42%,loss is 0.065803
after 5000 training steps, training accuracy on validate dataset is 98.08%,loss is 0.069263
after 6000 training steps, training accuracy on validate dataset is 98.18%,loss is 0.066537
after 7000 training steps, training accuracy on validate dataset is 98.44%,loss is 0.063063
after 8000 training steps, training accuracy on validate dataset is 98.46%,loss is 0.062138
after 9000 training steps, training accuracy on validate dataset is 98.48%,loss is 0.064127
training complete!Test accuracy is:  98.35


# RMSprop

In [64]:
myANN=annModel(layerDims,Xtrain,ytrain)
myANN.RMSpropOptimizer(0.001)

after 0 training steps, training accuracy on validate dataset is 24.44%,loss is 7.492584
after 1000 training steps, training accuracy on validate dataset is 97.22%,loss is 0.094276
after 2000 training steps, training accuracy on validate dataset is 98.04%,loss is 0.070703
after 3000 training steps, training accuracy on validate dataset is 98.16%,loss is 0.062427
after 4000 training steps, training accuracy on validate dataset is 98.24%,loss is 0.064905
after 5000 training steps, training accuracy on validate dataset is 97.96%,loss is 0.067854
after 6000 training steps, training accuracy on validate dataset is 98.18%,loss is 0.066930
after 7000 training steps, training accuracy on validate dataset is 98.42%,loss is 0.065285
after 8000 training steps, training accuracy on validate dataset is 98.1%,loss is 0.067979
after 9000 training steps, training accuracy on validate dataset is 98.24%,loss is 0.070708
training complete!Test accuracy is:  98.33
